In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1
)

template = PromptTemplate.from_template("{country_a}와 {country_b} 사이의 거리는 몇 km인가요? 당신 역할도 알려주세요.")
prompt = template.format(country_a="멕시코", country_b="태국")

chat.invoke(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 대략 16,600km 입니다. 두 나라는 지구 상에서 거의 정반대편에 위치해 있기 때문에 먼 거리를 이동해야 합니다. 따라서 비행기를 이용하여 이동하는 것이 보통입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 41, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaHSwoJTLe0aK1ZbSNr3J7nPYUiq3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2889c419-5ad5-49dc-b4e3-8152319ef92c-0', usage_metadata={'input_tokens': 41, 'output_tokens': 91, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "저는 {expert_class}입니다."),
    ("human", "{country_a}와 {country_b} 사이의 거리는 몇 km인가요? 당신의 역할도 알려주세요.")
])

prompt = template.format_messages(
    language="Korean",
    expert_class="지리 전문가",
    country_a="멕시코",
    country_b="태국"
)

chat.invoke(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 약 15,000 km입니다. 제 역할은 지리 전문가입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 76, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaHXGOiFA7HiFTq4GfaX6Atl2vnHD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c1c0fdd0-097e-4c54-8192-86c588fa72b6-0', usage_metadata={'input_tokens': 76, 'output_tokens': 38, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from langchain_core.output_parsers import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        return [s for i in text.split(",") if (s := i.strip())]
    
parser = CommaOutputParser()
parser.parse("hello, how, are,    ")

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 10,
    "question": "아시아, 북미에 있는 국가들은 무엇인가?"
})

['한국', '중국', '일본', '미국', '캐나다', '멕시코', '인도', '필리핀', '베트남']